In [1]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify
import seaborn as sns
import random
import numpy as np
import math
from sklearn.model_selection import train_test_split
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import tensorflow as tf
import pickle

# Importing Dataset

### Extracting image paths for SSID dataset

In [ ]:
dir = os.listdir('datasets/SIDD_Medium_Srgb/Data')

folders = []
GT = []
Noisy = []

for folder in dir:
    folders.append(folder)

for folder in folders:
    files = os.listdir('datasets/SIDD_Medium_Srgb/Data/' + folder)
    for image in files:
        if image[5] == 'G':
            GT.append('datasets/SIDD_Medium_Srgb/Data/' + folder + '/' + image)
        else:
            Noisy.append('datasets/SIDD_Medium_Srgb/Data/' + folder + '/' + image)

### Extracting image paths for RENOIR dataset

In [ ]:
dir = os.listdir('datasets/Mi3_Aligned')

folders = []
GT = []
Noisy = []

for folder in dir:
    folders.append(folder)

for folder in folders:
    added = False

    if folder == '.DS_Store':
        continue

    files = os.listdir('datasets/Mi3_Aligned/' + folder)

    for image in files:
        if 'Reference.bmp' in image:
            GT.append('datasets/Mi3_Aligned/' + folder + '/' + image)
        
        if 'Noisy.bmp' in image and not added:
            Noisy.append('datasets/Mi3_Aligned/' + folder + '/' + image)
            added = True

In [ ]:
df = pd.DataFrame()
df['Ground Truth Images'] = GT
df['Noisy Images'] = Noisy
df.head()

In [ ]:
df.shape

 # Exploratory Data Analysis

### Image Sizes

In [ ]:
size = []
for i in range(len(df)):
    img_gt = cv2.imread(df['Ground Truth Images'].iloc[i])
    size.append(img_gt.shape)

In [ ]:
df['image size'] = size
df['image size'] = df['image size'].astype(str)
df.head()

In [ ]:
fig = plt.figure(figsize=(100, 10))
y = list(df['image size'].value_counts())
x = df['image size'].value_counts().index.to_list()
plt.bar(x, y)
plt.title("Images vs Size")
plt.xlabel("Size of Images")
plt.ylabel("Number of Images")

### Displaying few clean-noisy image pairs

In [ ]:
sample = df.sample(3)
fig, ax = plt.subplots(len(sample), 2, figsize=(30, 30))
for i in range(len(sample)):
    img = cv2.imread(sample['Ground Truth Images'].iloc[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (512, 512))
    ax[i][0].imshow(img)
    ax[i][0].get_xaxis().set_visible(False)
    ax[i][0].get_yaxis().set_visible(False)
    ax[i][0].title.set_text("Ground Truth Images")

    img = cv2.imread(sample['Noisy Images'].iloc[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (512, 512))
    ax[i][1].imshow(img)
    ax[i][1].get_xaxis().set_visible(False)
    ax[i][1].get_yaxis().set_visible(False)
    ax[i][1].title.set_text("Noisy Images")

### Splitting the dataset into test and train

In [ ]:
X = df['Noisy Images']
y = df['Ground Truth Images']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# storing the image patches as pickle files
pickle.dump((X_train, y_train), open('train_path.pkl', 'wb'))
pickle.dump((X_test, y_test), open('test_path.pkl', 'wb'))

In [ ]:
X_train, y_train = pickle.load(open('train_path.pkl', "rb"))
X_test, y_test = pickle.load(open('test_path.pkl', 'rb'))

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

### Creating Patches

In [2]:
def patches(img, patch_size):
    patches = patchify(img, (patch_size, patch_size, 3), step=patch_size)
    return patches

In [ ]:
X_train_patches = []
y_train_patches = []

for i in range(len(X_train)):
    path = X_train.iloc[i]
    img_nsy = cv2.imread(path)
    img_nsy = cv2.cvtColor(img_nsy, cv2.COLOR_BGR2RGB)
    img_nsy = cv2.resize(img_nsy, (1024, 1024))
    patches_nsy = patches(img_nsy, 256)

    path = X_train.iloc[i]
    img_gt = cv2.imread(path)
    img_gt = cv2.cvtColor(img_gt, cv2.COLOR_BGR2RGB)
    img_gt = cv2.resize(img_gt, (1024, 1024))
    patches_gt = patches(img_gt, 256)

    rows = patches_nsy.shape[0]
    cols = patches_nsy.shape[1]

    for j in range(rows):
        for k in range(cols):
            X_train_patches.append(patches_nsy[j][k][0])
            y_train_patches.append(patches_gt[j][k][0])

X_train = np.array(X_train_patches)
y_train = np.array(y_train_patches)

In [ ]:
X_test_patches = []
y_test_patches = []

for i in range(len(X_test)):
    path = X_test.iloc[i]
    img_nsy = cv2.imread(path)
    img_nsy = cv2.cvtColor(img_nsy, cv2.COLOR_BGR2RGB)
    img_nsy = cv2.resize(img_nsy,(1024,1024))  #resizing the X_test images
    patches_nsy = patches(img_nsy,256)

    path = y_test.iloc[i]
    img_gt = cv2.imread(path)
    img_gt = cv2.cvtColor(img_gt, cv2.COLOR_BGR2RGB)
    img_gt = cv2.resize(img_gt,(1024,1024))  #resizing the y_test images
    patches_gt = patches(img_gt,256)

    rows = patches_nsy.shape[0]
    cols = patches_nsy.shape[1]

    for j in range(rows):
        for k in range(cols):
            X_test_patches.append(patches_nsy[j][k][0])
            y_test_patches.append(patches_gt[j][k][0])

X_test = np.array(X_test_patches)
y_test = np.array(y_test_patches)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
fig, axs = plt.subplots(2,5,figsize=(20,10))
r = random.sample(range(0, 2048), 5)

fig.suptitle('Train Image Patches',fontweight ="bold")
for i in range(5):
    axs[0][i].imshow(y_train[r[i]])
    axs[0][i].set_title('Ground Truth Image Patches')
    axs[1][i].imshow(X_train[r[i]])
    axs[1][i].set_title('Noisy Image Patches')

In [ ]:
fig, axs = plt.subplots(2,5,figsize=(20,10))
r = random.sample(range(0, 512), 5)

fig.suptitle('Test Image Patches',fontweight ="bold")
for i in range(5):
    axs[0][i].imshow(y_test[r[i]])
    axs[0][i].set_title('Ground Truth Image Patches')
    axs[1][i].imshow(X_test[r[i]])
    axs[1][i].set_title('Noisy Image Patches')

In [ ]:
print("Total number of image patches on train data : ", len(X_train))
print("Total number of image patches on test data : ", len(X_test))

In [ ]:
pickle.dump((X_train,y_train),open('train_data.pkl','wb'))
pickle.dump((X_test,y_test),open('test_data.pkl','wb'))

# Denoising few image patches using NLM filter

In [ ]:
fig, axis = plt.subplots(3, 4, figsize=(30, 15))
r = random.sample(range(0, rows), 4)
c = random.sample(range(0, cols), 4)
PSNR_nsy = []
PSNR_de_nsy = []

for i in range(4):
    img1 = patches_nsy[r[i]][c[i]][0]
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2 = patches_gt[r[i]][c[i]][0]
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

    axis[0][i].imshow(img1)
    axis[0][i].set_title('Noisy Image Patches')
    axis[1][i].imshow(img2)
    axis[1][i].set_title('GT Images Patches')

    # Denoising
    dst = cv2.fastNlMeansDenoisingColored(img1, None, 50, 10, 21, 7)
    axis[2][i].imshow(dst)
    axis[2][i].set_title('Denoised Image Patches')

    PSNR_nsy.append(psnr(img1, img2))
    PSNR_de_nsy.append(psnr(img1, dst))

In [ ]:
improvement = [x1 - x2 for(x1, x2) in zip(PSNR_de_nsy, PSNR_nsy)]

In [ ]:
from prettytable import PrettyTable
x = PrettyTable()
x.add_column("PSNR before denoising", PSNR_nsy)
x.add_column("PSNR after denoising", PSNR_de_nsy)
x.add_column("PSNR Improvement", improvement)
print(x)

# Creating Dataset for modeling using custom data generators in Keras

In [3]:
#Custom function to get denoised image prediction for noisy images
def prediction(img,model):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(1024,1024))
  img = img.astype("float32") / 255.0

  img_patches = patches(img,256)

  nsy=[]
  for i in range(4):
    for j in range(4):
      nsy.append(img_patches[i][j][0])
  nsy = np.array(nsy)

  pred_img = model.predict(nsy)
  pred_img = np.reshape(pred_img,(4,4,1,256,256,3))
  pred_img = unpatchify(pred_img, img.shape)
  return pred_img


#Custom function to get denoised image prediction for noisy images on quantized models using tflite
def prediction_tflite(img,model):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(1024,1024))
  img = img.astype("float32") / 255.0

  img_patches = patches(img,256)

  nsy=[]
  for i in range(4):
    for j in range(4):
      nsy.append(img_patches[i][j][0])
  nsy = np.array(nsy)
  pred=[]
  for patch in nsy:
    model.set_tensor(input_details[0]['index'], tf.expand_dims(patch,axis=0))
    model.invoke()
    tflite_model_predictions = model.get_tensor(output_details[0]['index'])
    pred.append(tflite_model_predictions)

  pred_img = np.reshape(pred,(4,4,1,256,256,3))
  pred_img = unpatchify(pred_img, img.shape)
  return pred_img

#Custom function to plot/visualize noisy, ground truth and predicted images
def visualize(sample,model):
  fig,ax = plt.subplots(len(sample),3,figsize=(30,30))
  for i in range(len(sample)):
    path = sample['Ground Truth Images'].iloc[i]
    test_img_gt = cv2.imread(path)
    test_img_gt = cv2.cvtColor(test_img_gt, cv2.COLOR_BGR2RGB)
    test_img_gt = cv2.resize(test_img_gt,(512,512))
    test_img_gt = test_img_gt.astype("float32") / 255.0
  
    path = sample['Noisy Images'].iloc[i]
    test_img_nsy = cv2.imread(path)
    pred_img = prediction(test_img_nsy,model)
    pred_img = cv2.resize(pred_img,(512,512))

    test_img_nsy = cv2.cvtColor(test_img_nsy, cv2.COLOR_BGR2RGB)
    test_img_nsy = cv2.resize(test_img_nsy,(512,512))
    test_img_nsy = test_img_nsy.astype("float32") / 255.0
    
    ax[i][0].imshow(test_img_nsy)
    ax[i][0].get_xaxis().set_visible(False)
    ax[i][0].get_yaxis().set_visible(False)
    ax[i][0].title.set_text("Noisy Image")

    ax[i][1].imshow(test_img_gt)
    ax[i][1].get_xaxis().set_visible(False)
    ax[i][1].get_yaxis().set_visible(False)
    ax[i][1].title.set_text("Ground Truth Image")

    ax[i][2].imshow(pred_img)
    ax[i][2].get_xaxis().set_visible(False)
    ax[i][2].get_yaxis().set_visible(False)
    ax[i][2].title.set_text("Predicted Image")

#Custom function that computes the psnr and ssim values for images
def psnr_and_ssim(X_test,y_test,model,model_type='Normal'):
  psnr_nsy = 0.0
  psnr_de_nsy = 0.0
  ssim_nsy = 0.0
  ssim_de_nsy = 0.0
  for i in range(len(X_test)):
    #getting the noisy images
    path = X_test.iloc[i]
    nsy = cv2.imread(path)  

    #getting the predicted images
    if model_type == 'Quantized': 
      pred = prediction_tflite(nsy,model)
    else:
      pred = prediction(nsy,model)

    #getting the ground truth images
    path = y_test.iloc[i]
    gt = cv2.imread(path)         
    gt = cv2.cvtColor(gt, cv2.COLOR_BGR2RGB)
    
    #Resizing the images
    gt = cv2.resize(gt,(1024,1024))
    nsy = cv2.resize(nsy,(1024,1024))

    #Normalizing the images
    gt = gt.astype("float32") / 255.0
    nsy = nsy.astype("float32") / 255.0

    #Computing psnr and ssim for test images
    psnr_nsy += psnr(gt,nsy)
    psnr_de_nsy += psnr(gt,pred)
    ssim_nsy += ssim(gt,nsy,multichannel=True,data_range=nsy.max() - nsy.min())
    ssim_de_nsy += ssim(gt,pred,multichannel=True,data_range=pred.max() - pred.min())

  psnr_nsy = psnr_nsy/len(X_test)
  psnr_de_nsy = psnr_de_nsy/len(X_test)
  ssim_nsy = ssim_nsy/len(X_test)
  ssim_de_nsy = ssim_de_nsy/len(X_test)
  return psnr_nsy, psnr_de_nsy,ssim_nsy,ssim_de_nsy

In [4]:
X_train_patches, y_train_patches = pickle.load(open('train_data.pkl', 'rb'))
X_test_patches,y_test_patches = pickle.load(open('test_data.pkl', 'rb'))

In [5]:
print(X_train_patches.shape)
print(y_train_patches.shape)
print(X_test_patches.shape)
print(y_test_patches.shape)

(4096, 256, 256, 3)
(4096, 256, 256, 3)
(1024, 256, 256, 3)
(1024, 256, 256, 3)


In [6]:
# Normalizing the image pixels
X_train_patches = X_train_patches.astype('float32') / 255.0
y_train_patches = y_train_patches.astype('float32') / 255.0
X_test_patches = X_test_patches.astype('float32') / 255.0
y_test_patches = y_test_patches.astype('float32') / 255.0

In [14]:
class Dataloader(tf.keras.utils.Sequence):
    def __init__(self, X, y, batch_size=1, shuffle=False):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(X))

    def __getitem__(self, i):
        batch_x = self.X[i * self.batch_size : (i + 1) * self.batch_size]
        batch_y= self.y[i * self.batch_size : (i + 1) * self.batch_size]
        return tuple((batch_x, batch_y))

    def __len__(self):
        return len(self.indexes)

    def on_epoch_end(self):
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

In [7]:
batch_size = 32
train_dataloader = Dataloader(X_train_patches,y_train_patches, batch_size, shuffle=True)
test_dataloader = Dataloader(X_test_patches,y_test_patches,batch_size, shuffle=True)

In [8]:
train_dataloader[0][0].shape

(32, 256, 256, 3)

# Baseline Model: Autoencoder

In [4]:
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Activation, Flatten, Dense, Input, MaxPooling2D, Add, Reshape, concatenate, AveragePooling2D, Multiply, GlobalAveragePooling2D, UpSampling2D, MaxPool2D,Softmax
from tensorflow.keras.activations import softmax
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.optimizers import Adam

In [ ]:
tf.keras.backend.clear_session()

input = Input(shape=(256, 256, 3))

# Encoder
x = Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(input)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)

# Decoder
x = Conv2DTranspose(128, (3, 3), strides=2, activation="relu", kernel_initializer="he_normal", padding="same")(x)
x = Conv2DTranspose(64, (3, 3), strides=2, activation="relu", kernel_initializer="he_normal", padding="same")(x)
x = Conv2DTranspose(32, (3, 3), strides=2, activation="relu", kernel_initializer="he_normal", padding="same")(x)
x = Conv2D(3, (3, 3), activation="sigmoid", kernel_initializer="he_normal", padding="same")(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(1e-03), loss=tf.keras.losses.MeanSquaredError())
autoencoder.summary()

In [2]:
print("Num GPU Avaliable: ", len(tf.config.list_physical_devices('GPU')))

Num GPU Avaliable:  1


In [ ]:
log_dir = 'logs/model_1'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_grads=True)
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, verbose=1, patience=2)
callback = [tensorboard, reducelr]

autoencoder.fit(train_dataloader, shuffle=True, epochs=20, validation_data=test_dataloader, callbacks=callback)

In [ ]:
%tensorboard --logdir "logs/model_1"

In [ ]:
autoencoder.save('models/autoencoder', save_format='tf')

In [ ]:
autoencoder = tf.keras.models.load_model('models/autoencoder.h5')

In [ ]:
sample = pd.DataFrame({'Ground Truth Images' : ['datasets/SIDD_Small_sRGB_Only/Data/0001_001_S6_00100_00060_3200_L/GT_SRGB_010.PNG','datasets/SIDD_Small_sRGB_Only/Data/0002_001_S6_00100_00020_3200_N/GT_SRGB_010.PNG','datasets/SIDD_Small_sRGB_Only/Data/0003_001_S6_00100_00060_3200_H/GT_SRGB_010.PNG'], 'Noisy Images' : ['datasets/SIDD_Small_sRGB_Only/Data/0001_001_S6_00100_00060_3200_L/NOISY_SRGB_010.PNG','datasets/SIDD_Small_sRGB_Only/Data/0002_001_S6_00100_00020_3200_N/NOISY_SRGB_010.PNG','datasets/SIDD_Small_sRGB_Only/Data/0003_001_S6_00100_00060_3200_H/NOISY_SRGB_010.PNG']})
#sample = pd.DataFrame({'Ground Truth Images' : ['datasets/SIDD_Medium_Srgb/Data/0073_003_IP_00200_01000_5500_L/0073_GT_SRGB_010.PNG'], 'Noisy Images' : ['datasets/SIDD_Medium_Srgb/Data/0073_003_IP_00200_01000_5500_L/0073_NOISY_SRGB_010.PNG']})
#sample = pd.DataFrame({'Ground Truth Images' : ['datasets/GT2.png'], 'Noisy Images' : ['datasets/N2.png']})

In [ ]:
visualize(sample, autoencoder)

path = sample['Ground Truth Images'].iloc[0]
test_img_gt = cv2.imread(path)
test_img_gt = cv2.cvtColor(test_img_gt, cv2.COLOR_BGR2RGB)
test_img_gt = cv2.resize(test_img_gt, (512, 512))
test_img_gt = test_img_gt.astype("float32") / 255.0

path = sample['Noisy Images'].iloc[0]
test_img_nsy = cv2.imread(path)

pred = prediction(test_img_nsy, autoencoder)
pred = cv2.resize(pred, (512, 512))

test_img_nsy= cv2.cvtColor(test_img_nsy, cv2.COLOR_BGR2RGB)
test_img_nsy = cv2.resize(test_img_nsy, (512, 512))
test_img_nsy = test_img_nsy.astype("float32") / 255.0

fig, ax = plt.subplots(1, 3, figsize=(30,30))
ax[0].imshow(test_img_nsy)
ax[0].title.set_text('Noisy')
ax[1].imshow(test_img_gt)
ax[1].title.set_text('GT')
ax[2].imshow(pred)
ax[2].title.set_text('Pred')
plt.show()


In [ ]:
psnr_nsy, psnr_de_nsy, ssim_nsy, ssim_de_nsy = psnr_and_ssim(X_test,y_test,autoencoder)
print('PSNR before denoising :', psnr_nsy)
print('PSNR after denoising :', psnr_de_nsy)
print('SSIM before denoising :', ssim_nsy)
print('SSIM after denoising :', ssim_de_nsy)

# RIDNet

In [5]:
class EAM(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        self.conv1 = Conv2D(64, (3, 3), dilation_rate=1, padding='same', activation='relu')
        self.conv2 = Conv2D(64, (3, 3), dilation_rate=2, padding='same', activation='relu')

        self.conv3 = Conv2D(64, (3, 3), dilation_rate=3, padding='same', activation='relu')
        self.conv4 = Conv2D(64, (3, 3), dilation_rate=4, padding='same', activation='relu')

        self.conv5 = Conv2D(64, (3, 3), padding='same', activation='relu')

        self.conv6 = Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv7 = Conv2D(64, (3, 3), padding='same')

        self.conv8 = Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv9 = Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv10 = Conv2D(64, (1, 1), padding='same')

        self.gap = GlobalAveragePooling2D()

        self.conv11 = Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv12 = Conv2D(64, (3, 3), padding='same', activation='sigmoid')
    
    def call(self, input):
        conv1 = self.conv1(input)
        conv1 = self.conv1(conv1)

        conv2 = self.conv3(input)
        conv2 = self.conv4(conv2)

        concat = concatenate([conv1, conv2])
        conv3 = self.conv5(concat)
        add1 = Add()([input, conv3])

        conv4 = self.conv6(add1)
        conv4 = self.conv7(conv4)
        add2 = Add()([conv4, add1])
        add2 = Activation('relu')(add2)

        conv5 = self.conv8(add2)
        conv5 = self.conv9(conv5)
        conv5 = self.conv10(conv5)
        add3 = Add()([add2, conv5])
        add3 = Activation('relu')(add3)

        gap = self.gap(add3)
        gap = Reshape((1, 1, 64))(gap)
        conv6 = self.conv11(gap)
        conv6 = self.conv11(conv6)

        mul = Multiply()([conv6, add3])
        out = Add()([input, mul])

        return out

In [ ]:
tf.keras.backend.clear_session()

input = Input(shape=(256, 256, 3))

conv1 = Conv2D(64, (3, 3), padding='same')(input)
eam1 = EAM()(conv1)
eam2 = EAM()(eam1)
eam3 = EAM()(eam2)
eam4 = EAM()(eam3)
conv2 = Conv2D(3, (3, 3), padding='same')(eam4)
out = Add()([conv2, input])

RIDNet = Model(input, out)
RIDNet.compile(optimizer=tf.keras.optimizers.Adam(1e-03), loss=tf.keras.losses.MeanSquaredError())
RIDNet.summary()


In [ ]:
batch_size=8
train_dataloader = Dataloader(X_train_patches,y_train_patches, batch_size, shuffle=True)
test_dataloader = Dataloader(X_test_patches,y_test_patches,batch_size, shuffle=True)

In [ ]:
log_dir="logs/model_4"
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,verbose=1,patience=4,min_delta=0.00001)
callback = [tensorboard,reducelr]
RIDNet.fit(train_dataloader,shuffle=True,epochs=20,validation_data= test_dataloader, callbacks=callback)

In [7]:
RIDNet = tf.keras.models.load_model('models/RIDNetGH.h5', custom_objects={'EAM': EAM})

ImportError: Filepath looks like a hdf5 file but h5py is not available. filepath=models/RIDNetGH.h5

In [ ]:
visualize(sample, RIDNet)

In [ ]:
RIDNet.save('models/RIDNet.tf')

# CBDNet

In [11]:
tf.keras.backend.clear_session()

input = Input(shape=(256, 56, 3))

# Noise estimation subnetwork
x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(input)
x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
x = Conv2D(3, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)

# Non Blind denoising subnetwork
x = concatenate([x, input])
conv1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)

pool1 = AveragePooling2D(pool_size=(2, 2), padding='same')(conv2)
conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
conv5 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)

pool2 = AveragePooling2D(pool_size=(2, 2), padding='same')(conv5)
conv6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
conv7 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)
conv8 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
conv9 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv8)
conv10 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv9)
conv11 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv10)

upsample1 = Conv2DTranspose(128, (3, 3), strides=2, activation="relu", kernel_initializer='he_normal',padding="same")(conv11)
add1 = Add()([upsample1,conv5])
conv12 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(add1)
conv13 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv12)
conv14 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv13)

upsample2 = Conv2DTranspose(64, (3, 3), strides=2, activation="relu", kernel_initializer='he_normal',padding="same")(conv14)
add1 = Add()([upsample2, conv2])
conv15 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(add1)
conv16 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv15)

out = Conv2D(3, (1, 1), kernel_initializer='he_normal', padding='same')(conv16)
out = Add()([out, input])

CBDNet = Model(input, out)
CBDNet.compile(optimizer=tf.keras.optimizers.Adam(1e-03), loss=tf.keras.losses.MeanSquaredError())
CBDNet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 56, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 256, 56, 32)  896         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 56, 32)  9248        ['conv2d[0][0]']                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 256, 56, 32)  9248        ['conv2d_1[0][0]']           

In [12]:
log_dir="/logs/cbdnet"

tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,verbose=1,patience=4,min_delta=0.00001)
callback = [tensorboard,reducelr]
CBDNet.fit(train_dataloader,shuffle=True,epochs=30,validation_data= test_dataloader,callbacks=callback)

Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'model/add/add' defined at (most recent call last):
    File "c:\Software\Programs\Conda\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Software\Programs\Conda\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Software\Programs\Conda\envs\tf\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Software\Programs\Conda\envs\tf\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Software\Programs\Conda\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 737, in start
      self.io_loop.start()
    File "c:\Software\Programs\Conda\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\Software\Programs\Conda\envs\tf\lib\asyncio\base_e